In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import datetime
from tqdm.notebook import tqdm
import re

In [ ]:
# BOX OFFICE 연간 Weekly 데이터 추출

set_year = input("연도 입력 : ")
now = datetime.datetime.now()

URL = 'https://www.boxofficemojo.com/weekly/by-year/'+set_year+'/'

headers = { 
'User-Agent' : ('Mozilla/5.0 (Windows NT 10.0;Win64; x64)\
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98\
Safari/537.36')}

resq = requests.get(URL, headers=headers)

fin_data = []

Status = True
while Status:
    if resq.status_code != 200:
        time.sleep(2)
        resq = requests.get(URL, headers=headers)
    else:
        Status = False
    
soup = BeautifulSoup(resq.text, 'html.parser')

data = soup.find_all('td', {'class':True})

result = [data[i * 12:(i+1)*12] for i in range((len(data)+11)//12)]

for i in tqdm(range(len(result))):
    Top10Gross = result[i][1].get_text()
    LW1 = result[i][2].get_text()
    OverallGross = result[i][3].get_text()
    LW2 = result[i][4].get_text()
    Release1 = result[i][5].get_text()
    Release2 = result[i][6].get_text()
    Week = result[i][10].get_text()
    reg_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    week_list = {'week':Week, 'year':set_year, 'top_gross':Top10Gross, 'top_lw':LW1, 'overall_gross':OverallGross, 'overall_lw':LW2,
                'releases':Release1, '#1 Release':Release2, 'top_tmdb_id':None, 'regist_date':reg_date}

    fin_data.append(week_list)
        
df = pd.DataFrame(fin_data, columns=['week', 'year', 'top_gross', 'top_lw', 'overall_gross',  'overall_lw', 'releases', '#1 Release','top_tmdb_id', 'regist_date'])
# df.drop_duplicates('#1 Release')

filename = set_year + '년 데이터_' + 'boxoffice_result_%s-%s-%s  %s시 %s분 %s초.xlsx' %(now.year, now.month, now.day, now.hour, now.minute, now.second)

url_preserve = pd.ExcelWriter(filename, options={'strings_to_urls': False})
df.to_excel(url_preserve)
url_preserve.save()
print('Success')

In [ ]:
# 각 주차마다 세부 데이터 추출

link_data = soup.find_all('a', {'class':'a-link-normal'})
clean_data = []

for x in range(0, len(link_data)):
    if "weekly/20" in link_data[x]['href']:
        clean_data.append(link_data[x]['href'])
        
set_list = set(clean_data)
clean = list(set_list)

d_fin = []

for i in tqdm(range(0, len(clean))):
        
    d_URL = 'https://www.boxofficemojo.com' + clean[i]
    
    d_resq = requests.get(d_URL, headers=headers)
    
    Status = True
    while Status:
        if (d_resq.status_code != 200):
            time.sleep(2)
            d_resq = requests.get(d_URL, headers=headers)
        else:
            Status = False
    
    d_soup = BeautifulSoup(d_resq.text, 'html.parser')
    
    d_data = d_soup.find_all('td', {'class' : True})
    
    d_result = [d_data[i * 13:(i+1)*13] for i in range((len(d_data)+12)//13)]
    
    d_week = len(clean)-i
    
    for j in range(0, len(d_result)):
        d_title = d_result[j][2].get_text()
        d_url = 'https://www.boxofficemojo.com' + clean[i]
        d_gross = d_result[j][3].get_text()
        d_average = d_result[j][7].get_text()
        d_total_gross = d_result[j][8].get_text()
        reg_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # imdb_id 구하기
        d_r_req = requests.get('https://www.boxofficemojo.com' + d_result[j][2].find('a')['href'], headers=headers)
        Status = True
        while Status:
            if (d_r_req.status_code != 200):
                time.sleep(2)
                d_r_req = requests.get('https://www.boxofficemojo.com' + d_result[j][2].find('a')['href'], headers=headers)
            else:
                Status = False
        d_r_soup = BeautifulSoup(d_r_req.text, 'html.parser')
        
        if d_r_soup.find_all('div', {'id':'title-summary-refiner'}) != None:
            imdb_id = ''.join(re.findall('e\/(.+?)\/\?', str(d_r_soup.find('div', {'id':'title-summary-refiner'}))))
        else:
            imdb_id = None
        
        d_result_data = {'tmdb_id':None, 'imdb_id':imdb_id, 'title':d_title,
                    'url':d_url, 'series_id':None, 'week':d_week, 'year':set_year, 'gross':d_gross,
                    'average':d_average, 'total_gross':d_total_gross, 'regist_date':reg_date}
        d_fin.append(d_result_data)
    
d_df = pd.DataFrame(d_fin, columns=['tmdb_id', 'imdb_id', 'title', 'url', 'series_id',  'week', 'year', 'gross','average','total_gross', 'regist_date'])

d_filename = set_year + '년 데이터_' + 'boxoffice_detail_result_%s-%s-%s  %s시 %s분 %s초.xlsx' %(now.year, now.month, now.day, now.hour, now.minute, now.second)

d_url_preserve = pd.ExcelWriter(d_filename, options={'strings_to_urls': False})
d_df.to_excel(d_url_preserve)
d_url_preserve.save()
print('Done')